# Class magic methods and inheritance

## 1. Magic methods
Let's create a class to represent money - an amount in a given currency - on which we can do many operations, just like numbers (floats and ints). 

In [1]:
class Money:
    def __init__(self, amount, currency="USD"):
        self.amount   = float(amount)
        self.currency = currency
    
    #how the money instance is cast to a string
    def __str__(self):
        return "{:.2f}{}".format(self.amount, self.currency)
    
    #how the money instance is cast to an integer
    def __int__(self): 
        return int(self.amount)
    
    #how the money instance is cast to a float
    def __float__(self):
        return float(self.amount)
    
print(str(Money(10)))
print(int(Money(10)))
print(float(Money(10)))

10.00USD
10
10.0


### 1.1. Comparison methods

In [2]:
class Money: 
    def __init__(self, amount, currency):
        self.amount   = float(amount)
        self.currency = currency
        
    def __eq__(self, other): 
        if isinstance(other, Money): 
            return self.amount == other.amount and self.currency == other.currency
        if isinstance(other, (float, int)):
            return self.amount == other
        raise TypeError("Cannot compare money with {}".format(type(other)))
        
    def __neq__(self, other): 
        return not self == other
        
    def __gt__(self, other): 
        if isinstance(other, Money):
            if self.currency != other.currency: 
                raise TypeError("Cannot compare {} with {}".format(self.currency, other.currency))
            return self.amount > other.amount
        if isinstance(other, (float, int)):
            return self.amount == other
        raise TypeError("Cannot compare money with {}".format(type(other)))
        
    def __ge__(self, other):
        return self > other or self == other
    
    def __lt__(self, other): 
        return not self > other and self != other
    
    def __le__(self, other): 
        return self < other or self == other
    
#let's try it out! 
print(Money(10, "EUR") == 10)
print(Money(10, "EUR") != Money(20, "EUR"))
print(Money(10, "EUR") > Money(5, "EUR"))
print(Money(10, "EUR") <= Money(30, "EUR"))

#let's make sure we get the expected errors
try: 
    print(Money(10, "USD") > Money(200, "JPY"))
except TypeError as e: 
    print(e)

True
True
True
True
Cannot compare USD with JPY


### 1.2. Mathematical methods

In [3]:
class Money:
    def __init__(self, amount, currency="USD"):
        self.amount   = float(amount)
        self.currency = currency.strip()
        
    def __str__(self):
        return "{:.2f} {}".format(self.amount, self.currency)
        
    def __add__(self, other):
        if isinstance(other, (float, int)): 
            return Money(self.amount + other, self.currency)
        if isinstance(other, Money): 
            if other.currency == self.currency:
                return Money(self.amount + other.amount, self.currency)
            raise TypeError("You can only add two money amounts of the same currency")
        raise TypeError("Unable to add money to {}".format(type(other)))
        
    def __sub__(self, other):
        if isinstance(other, (float, int)): 
            return Money(self.amount - other, self.currency)
        if isinstance(other, Money): 
            if other.currency == self.currency: 
                return Money(self.amount - other.amount, self.currency)
            raise TypeError("You can only add currencies of the same currency")
        raise TypeError("Unable to subtract {} from money".format(type(other)))
        
    def __mul__(self, other):
        if isinstance(other, (float, int)): 
            return Money(self.amount * other, self.currency)
        raise TypeError("Unable to multiply money by {}".format(type(other)))
        
    def __truediv__(self, other):
        if isinstance(other, (float, int)): 
            if other == 0: 
                raise ZeroDivisionError("Can divide money by 0")
            return Money(self.amount / other, self.currency)
        raise TypeError("Unable to divide money by {}".format(type(other)))

In [4]:
#Let's try it out
print(Money(10) + 20)
print(Money(10, "USD") + Money(10, "USD"))
print(Money(10, "USD") * 10)
print(Money(10, "USD") / 4)

#let's try adding two different currencies
try: 
    Money(10, "USD") + Money(40, "EUR")
except Exception as e: 
    print(e)

30.00 USD
20.00 USD
100.00 USD
2.50 USD
You can only add two money amounts of the same currency


In [5]:
print(Money(1)+1)

#Spoiler: this will fail
print(1 + Money(1))

2.00 USD


TypeError: unsupported operand type(s) for +: 'int' and 'Money'

In [6]:
#we need to add the symetric magic methods
class Money:
    def __init__(self, amount, currency="USD"):
        self.amount   = float(amount)
        self.currency = currency.strip()
        
    def __str__(self):
        return "{:.2f} {}".format(self.amount, self.currency)
        
    def __add__(self, other):
        if isinstance(other, (float, int)): 
            return Money(self.amount + other, self.currency)
        if isinstance(other, Money): 
            if other.currency == self.currency:
                return Money(self.amount + other.amount, self.currency)
            raise TypeError("You can only add two money amounts of the same currency")
        raise TypeError("Unable to add money to {}".format(type(other)))
        
    def __radd__(self, other):
        #called when something + Money rather than Money + something
        #Let's simply do the reverse
        return self + other
    
    def __sub__(self, other):
        if isinstance(other, (float, int)): 
            return Money(self.amount - other, self.currency)
        if isinstance(other, Money): 
            if other.currency == self.currency: 
                return Money(self.amount - other.amount, self.currency)
            raise TypeError("You can only add currencies of the same currency")
        raise TypeError("Unable to subtract {} from money".format(type(other)))
        
    def __rsub__(self, other):
        #called when something - Money rather than Money - something
        #Let's simply do the reverse
        return self-other
        
    def __mul__(self, other):
        if isinstance(other, (float, int)): 
            return Money(self.amount * other, self.currency)
        raise TypeError("Unable to multiply money by {}".format(type(other)))
        
    def __rmul__(self, other):
        #called when something * Money rather than Money * something
        #Let's simply do the reverse
        return self * other
    
    def __truediv__(self, other):
        if isinstance(other, (float, int)): 
            if other == 0: 
                raise ZeroDivisionError("Can divide money by 0")
            return Money(self.amount / other, self.currency)
        raise TypeError("Unable to divide money by {}".format(type(other)))
        
    #we won't implement the __rtruediv__
    #it wouldn't make sense to divide a number by Money
    
#let's try it out! 
print(Money(10) + 20)
print(Money(10, "USD") + Money(10, "USD"))
print(Money(10, "USD") * 10)
print(Money(10, "USD") / 4)
print(1 + Money(10))
print(3 * Money(20))
print(3 - Money(30))

30.00 USD
20.00 USD
100.00 USD
2.50 USD
11.00 USD
60.00 USD
27.00 USD


### 1.3. Container methods

In [8]:
#recall the calendar class we created earlier in the course
class Calendar: 
    def __init__(self, dates):
        self.dates = sorted(list(dates))
        
    def __getitem__(self, index): 
        return self.dates[index]
    
    def __setitem__(self, index, value): 
        self.dates.remove(self.dates[index])
        if value not in self.dates: 
            self.dates.append
            self.dates.sort()
        
    def __contains__(self, date):
        return date in self.dates
    
    def __len__(self): 
        return len(self.dates)
    
#let's try it out
from datetime import date, timedelta
import random

#let's create some unique dates
datelist = set(date.today() + timedelta(random.randint(-100, 100)) for i in range(50))

#create our instance of calendar
calendar = Calendar(datelist)

print(calendar[0])
print(calendar[-1])
print(len(calendar))
print(date.today() in calendar)

2019-02-07
2019-08-25
43
False


## 2. Inheritence

In [9]:
#Dogs and cats are both animals
#they share some traits (such as eating...) and have some of specific features
class Animal:
    def __init__(self, name):
        self.name = name
        self.weight = 2
        
    def eat(self): 
        self.weight += 1
        return self
        
class Cat(Animal):
    def meow(self): 
        return "Meowwwww... Grrrrrrr..."
        
class Dog(Animal):
    def bark(self):
        return "Woof woof!"

tom = Cat("Tom")
tom.eat()
tom.eat()
print(tom.weight)
print(tom.meow())

bo = Dog("Bo")
bo.eat()
bo.eat()
bo.eat()
bo.eat()

print(bo.weight)
print(bo.bark())

#yes, tom is an animal, but not a dog
print(isinstance(tom, Animal))
print(isinstance(tom, Dog))

4
Meowwwww... Grrrrrrr...
6
Woof woof!
True
False


### 2.1 Calling the `super` parent method: 

In [10]:
class Instrument: 
    def __init__(self, ticker, assetclass):
        self.ticker = ticker
        self.assetclass = assetclass
        
    def __str__(self): 
        return "{} ({})".format(self.ticker, type(self).__name__)
        
class Stock(Instrument):
    def __init__(self, ticker):
        super(Stock, self).__init__(ticker, "Equity")
        
class Future(Instrument):
    def __init__(self, ticker, underlying, expiration, size):
        super(Future, self).__init__(ticker, "Future")
        self.underlying = underlying
        self.expiration = expiration
        self.size = size
        
print(Stock("GLE"))
print(Future("ESZ9", "SPX", date(2019, 12, 31), 1000))

GLE (Stock)
ESZ9 (Future)


### 2.2. Creating custom exceptions

In [11]:
class LanguageException(Exception): 
    pass

class GrammarMistake(LanguageException):
    pass

class SpellingMistake(LanguageException): 
    pass

def correct(text):
    #do something to the text
    if "and me are" in text: 
        raise GrammarMistake("Try 'and I' instead of 'and me'")
    if "freind" in text: 
        raise SpellingMistake("Try 'friend' instead of 'freind'")
    return text

try: 
    correct("My dad and me are going to the cinema tonight")
except GrammarMistake as e: 
    print(e)
    
try: 
    correct("My girlfreind and I are going to the cinema tonight")
except SpellingMistake as e: 
    print(e)

Try 'and I' instead of 'and me'
Try 'friend' instead of 'freind'
